In [2]:
import math
import random
import time
import numpy as np

In [9]:
def american_to_percentage(odds):
    if odds >= 100:
        return 100 / (odds + 100)
    elif odds <= -100:
        return abs(odds) / (abs(odds) + 100)
    else:
        print("Invalid Number, cannot be between -100 and 100")
        return None
    
def percentage_to_american(probability):
    if probability == 0:
        return None
    elif probability > 0.5:
        return round((probability * 100) / (1 - probability) * -1)
    else:
        return round((100 / probability) - 100)
    
def calculate_winnings(odds, bet_amount):
    if odds > 0:
        winnings = (odds / 100) * bet_amount
    else:
        winnings = (100 / abs(odds)) * bet_amount
    
    return round(winnings, 2)

def parlay(odds_list):
    parlay_probability = 1
    for odds in odds_list:
        parlay_probability *= american_to_percentage(odds)
    return parlay_probability   
    

In [16]:
def simulate_betting(
    odds=-110,
    paid_odds=-110,
    goal=200,
    initial_balance=100,
    bet_percentage=0.025,
    iterations=10_000,
    max_bets=1_000_000,
):
    win = 0
    lost = 0
    bets_to_goal = (
        []
    )  # List to store the number of bets to reach the goal for each successful session

    for _ in range(iterations):
        current_balance = initial_balance
        bets = 0

        while 0 < current_balance < goal and bets < max_bets:
            bets += 1
            bet_amount = (
                current_balance * bet_percentage
            )  # Bet a percentage of the current balance

            # Ensure the bet does not exceed the current balance
            if bet_amount > current_balance:
                bet_amount = current_balance

            # Deduct bet from current balance
            current_balance -= bet_amount

            # Check if the bet is won
            if random.random() < american_to_percentage(odds):
                # Winning outcome
                winnings = calculate_winnings(paid_odds, bet_amount)
                current_balance += winnings + bet_amount
            else:
                # Losing outcome; bet remains a percentage of the new balance next round
                pass

            # Break if goal is reached
            if current_balance >= goal:
                bets_to_goal.append(
                    bets
                )  # Track number of bets for this successful session
                break

        # Track wins and losses
        if current_balance >= goal:
            win += 1
        else:
            lost += 1

    # Calculate average bets to reach the goal for successful sessions
    average_bets_to_goal = sum(bets_to_goal) / len(bets_to_goal) if bets_to_goal else 0

    # Return results as a tuple
    return win / iterations, average_bets_to_goal


# Simulate the batch betting strategy
def simulate_batch_betting(
    odds=-110,
    paid_odds=-110,
    goal=200,
    initial_balance=100,
    bet_percentage=0.025,
    bets_per_batch=10,
    iterations=10_000,
    max_bets=1_000_000,
):
    win = 0
    lost = 0
    bets_to_goal = []

    for _ in range(iterations):
        current_balance = initial_balance
        bets = 0

        while 0 < current_balance < goal and bets < max_bets:
            bets += 1
            bet_amount = (
                current_balance * bet_percentage
            )  # Bet a percentage of the current balance
            bet_amount = min(
                bet_amount, current_balance
            )  # Ensure bet doesn't exceed balance

            # Perform a batch of bets
            batch_winnings = 0
            for _ in range(bets_per_batch):
                if random.random() < american_to_percentage(odds):
                    batch_winnings += calculate_winnings(paid_odds, bet_amount) + bet_amount

            # Update balance with winnings (or losses)
            current_balance += batch_winnings - bet_amount * bets_per_batch

            # Break if the goal is reached
            if current_balance >= goal:
                bets_to_goal.append(bets)
                break

        # Track wins and losses
        if current_balance >= goal:
            win += 1
        else:
            lost += 1

    # Calculate average bets to reach the goal for successful sessions
    average_bets_to_goal = sum(bets_to_goal) / len(bets_to_goal) if bets_to_goal else 0

    # Return results as a tuple
    return win / iterations, average_bets_to_goal

In [20]:
def find_optimal_bet_percentage(
    odds=-110,
    paid_odds=-110,
    goal=200,
    initial_balance=100,
    iterations=10_000,
    max_bets=1_000_000,
):
    best_bet_percentage = None
    best_success_rate = 0
    best_average_bets = float("inf")

    bet_percentages_to_test = [
        x * 0.005 for x in range(1, 21)
    ]  # Testing bet percentages from 0.005 to 0.1

    for bet_percentage in bet_percentages_to_test:
        print(f"Testing bet percentage: {bet_percentage}")
        success_rate, avg_bets = simulate_betting(
            odds=odds,
            paid_odds=paid_odds,
            goal=goal,
            initial_balance=initial_balance,
            bet_percentage=bet_percentage,
            iterations=iterations,
            max_bets=max_bets,
        )

        print(
            f"Success rate: {success_rate:.4f}, Average Bets to Reach Goal: {avg_bets:.2f}"
        )

        # Determine if this is the best bet percentage based on success rate and avg bets
        if success_rate > best_success_rate or (
            success_rate == best_success_rate and avg_bets < best_average_bets
        ):
            best_success_rate = success_rate
            best_average_bets = avg_bets
            best_bet_percentage = bet_percentage

    print("\nOptimal Bet Percentage Found:")
    print(
        f"Bet Percentage: {best_bet_percentage:.4f}, Success Rate: {best_success_rate:.4f}, Average Bets: {best_average_bets:.2f}"
    )


# Function to find the optimal bet percentage with batch betting strategy
def find_optimal_batch_bet_percentage(
    odds=-110,
    paid_odds=-110,
    goal=200,
    initial_balance=100,
    iterations=10_000,
    max_bets=1_000_000,
    bets_per_batch=10,
):
    best_bet_percentage = None
    best_success_rate = 0
    best_average_bets = float("inf")

    bet_percentages_to_test = [
        x * 0.005 for x in range(3, 21)
    ]  # Testing bet percentages from 0.015 to 0.1

    for bet_percentage in bet_percentages_to_test:
        print(f"Testing bet percentage: {bet_percentage}")
        success_rate, avg_bets = simulate_batch_betting(
            odds=odds,
            paid_odds=paid_odds,
            goal=goal,
            initial_balance=initial_balance,
            bet_percentage=bet_percentage,
            bets_per_batch=bets_per_batch,
            iterations=iterations,
            max_bets=max_bets,
        )

        print(
            f"Success rate: {success_rate:.4f}, Average Bets to Reach Goal: {avg_bets:.2f}"
        )

        # Determine if this is the best bet percentage based on success rate and avg bets
        if success_rate > best_success_rate or (
            success_rate == best_success_rate and avg_bets < best_average_bets
        ):
            best_success_rate = success_rate
            best_average_bets = avg_bets
            best_bet_percentage = bet_percentage

    print("\nOptimal Bet Percentage Found:")
    print(
        f"Bet Percentage: {best_bet_percentage:.4f}, Success Rate: {best_success_rate:.4f}, Average Bets: {best_average_bets:.2f}"
    )

In [15]:
print(calculate_winnings(-110, 2.5))

2.27


In [22]:
# find_optimal_bet_percentage(
#     odds=-130, paid_odds=-110, goal=200, initial_balance=100, iterations=100_000
# )

find_optimal_batch_bet_percentage(
    odds=-140,
    paid_odds=-110,
    goal=2,
    initial_balance=1,
    iterations=10_000,
    max_bets=1_000_000,
    bets_per_batch=10,  # Place 10 bets at once
)

Testing bet percentage: 0.015
Success rate: 1.0000, Average Bets to Reach Goal: 136.76
Testing bet percentage: 0.02
Success rate: 1.0000, Average Bets to Reach Goal: 39.50
Testing bet percentage: 0.025
Success rate: 1.0000, Average Bets to Reach Goal: 32.59
Testing bet percentage: 0.03
Success rate: 1.0000, Average Bets to Reach Goal: 24.96
Testing bet percentage: 0.035
Success rate: 1.0000, Average Bets to Reach Goal: 23.00
Testing bet percentage: 0.04
Success rate: 1.0000, Average Bets to Reach Goal: 20.22
Testing bet percentage: 0.045
Success rate: 1.0000, Average Bets to Reach Goal: 18.74
Testing bet percentage: 0.05
Success rate: 1.0000, Average Bets to Reach Goal: 16.68
Testing bet percentage: 0.055
Success rate: 0.9998, Average Bets to Reach Goal: 16.34
Testing bet percentage: 0.06
Success rate: 0.9999, Average Bets to Reach Goal: 15.86
Testing bet percentage: 0.065
Success rate: 0.9998, Average Bets to Reach Goal: 14.39
Testing bet percentage: 0.07


KeyboardInterrupt: 